 # Normalizing Flows - Introduction (Part 1)
 ## Introduction
 In standard probabilistic modeling practice, we represent our beliefs over unknown continuous quantities with simple parametric distributions like the normal, exponential, and Laplacian distributions (or a mixture of these). However, using such simple forms, which are commonly symmetric and unimodal (or have a fixed number of modes), restricts the performance and flexibility of our methods. For instance, standard variational inference in the Variational Autoencoder uses independent normal distributions to represent the variational family. The true posterior is neither independent nor normally distributed, which results in suboptimal inference and simplifies the model that is learnt. In other scenarios, we are likewise restricted by not being able to model multimodal distributions and heavy or light tails.

 This series of tutorials explains Normalizing Flows, a family of methods for constructing flexible learnable probability distributions, and shows how you can use the NF library of Pyro for learning complex models and for flexible variational inference. In the first tutorial, we introduce the main idea of Normalizing Flows (NFs) and demonstrate learning simple univariate distributions. In two forthcoming tutorials, we will reproduce state-of-the-art results for multivariate density estimation on tabular and image datasets, as well as describe how to use NFs for SVI in Pyro. In a final tutorial, we explain how to add a new flow method to Pyro and the process of making an independent contribution to the project.

 ## Univariate Distributions
 ### Background
 Normalizing Flows are a family of methods for constructing flexible distributions. Let's first restrict our attention to representing univariate distributions. The basic idea is that a simple source of noise, for example a variable with a standard normal distribution, $X\sim\mathcal{N}(0,1)$, is passed through a bijective (i.e. invertible) function, $g(\cdot)$ to produce a more complex transformed variable $Y=g(X)$.

 For a given random variable, we typically want to perform two operations: sampling and scoring. Sampling $Y$ is trivial. First, we sample $X=x$, then calculate $y=g(x)$. Scoring $Y$, or rather, evaluating the log-density $\ln(p_Y(y))$, is more involved. How does the density of $Y$ relate to the density of $X$? We can use the substitution rule of integral calculus to answer this. Suppose we want to evaluate the expectation of some function of $X$. Then,
 \begin{align}
     \mathbb{E}_{p_X(\cdot)}\left[f(X)\right] &= \int_{\text{supp}(X)}f(x)p_X(x)dx\\
     &= \int_{\text{supp}(Y)}f(g^{-1}(y))p_X(g^{-1}(y))\left|\frac{dx}{dy}\right|dy\\
     &= \mathbb{E}_{p_Y(\cdot)}\left[f(g^{-1}(Y))\right],
 \end{align}
 where $\text{supp}(X)$ denotes the support of $X$, which in this case is $(-\infty,\infty)$. Crucially, we used the fact that $g$ is bijective to apply the substitution rule in going from the first to the second line. Equating the last two lines we get,
 \begin{align}
     \ln(p_Y(y)) &= \ln(p_X(g^{-1}(y)))+\ln\left(\left|\frac{dx}{dy}\right|\right)\\
     &= \ln(p_X(g^{-1}(y)))-\ln\left(\left|\frac{dy}{dx}\right|\right).
 \end{align}
 Inituitively, this equation says that the density of $Y$ is equal to the density at the corresponding point in $X$ plus a term that corrects for the warp in volume around an infinitesimally small length around $Y$ caused by the transformation.

 If $g$ is cleverly constructed (and we will see several examples shortly), we can produce distributions that are more complex than standard normal noise and yet have easy sampling and computationally tractable scoring. Moreover, we can compose such bijective transformations to produce even more complex distributions. By an inductive argument, if we have $L$ transforms $g_{(0)}, g_{(1)},\ldots,g_{(L-1)}$, then the log-density of the transformed variable $Y=(g_{(0)}\circ g_{(1)}\circ\cdots\circ g_{(L-1)})(X)$ is
 \begin{align}
     \ln(p_Y(y)) &= \ln\left(p_X\left(\left(g_{(L-1)}^{-1}\circ\cdots\circ g_{(0)}^{-1}\right)\left(y\right)\right)\right)+\sum^{L-1}_{l=0}\ln\left(\left|\frac{dg^{-1}_{(l)}(y_{(l)})}{dy'}\right|\right),
     %\left( g^{(l)}(y^{(l)})
     %\right).
 \end{align}
 where we've defined $y_{(0)}=x$, $y_{(L-1)}=y$ for convenience of notation.

 In a latter section, we will see how to generalize this method to multivariate $X$. The field of Normalizing Flows aims to construct such $g$ for multivariate $X$ to transform simple i.i.d. standard normal noise into complex, learnable, high-dimensional distributions. The methods have been applied to such diverse applications as image modeling, text-to-speech, unsupervised language induction, data compression, and modeling molecular structures. As probability distributions are the most fundamental component of probabilistic modeling we will likely see many more exciting state-of-the-art applications in the near future.

 ### Fixed Univariate Transforms in Pyro
 PyTorch contains classes for representing *fixed* univariate bijective transformations, and sampling/scoring from transformed distributions derived from these. Pyro extends this with a comprehensive library of *learnable* univariate and multivariate transformations using the latest developments in the field. As Pyro imports all of PyTorch's distributions and transformations, we will work solely with Pyro. We also note that the NF components in Pyro can be used independently of the probabilistic programming functionality of Pyro, which is what we will be doing in the first two tutorials.

 Let us begin by showing how to represent and manipulate a simple transformed distribution,
 \begin{align}
 X &\sim \mathcal{N}(0,1)\\
 Y &= \text{exp}(X).
 \end{align}
 You may have recognized that this is by definition, $Y\sim\text{LogNormal}(0,1)$.

 We begin by importing the relevant libraries:

In [ ]:
import torch
import pyro
import pyro.distributions as dist
import pyro.distributions.transforms as T
import matplotlib.pyplot as plt
import seaborn as sns


 A variety of bijective transformations live in the `pyro.distributions.transforms` module, and the classes to define transformed distributions live in `pyro.distributions`. We first create the base distribution of $X$ and the class encapsulating the transform $\text{exp}(\cdot)$:

In [ ]:
dist_x = dist.Normal(torch.zeros(1), torch.ones(1))
exp_transform = T.ExpTransform()


 The class `ExpTransform` derives from `torch.distributions.transforms.Transform` and defines the forward, inverse, and log-absolute-derivative operations for this transform,
 \begin{align}
     g(x) &= \text{exp(x)}\\
     g^{-1}(y) &= \ln(y)\\
     \ln\left(\left|\frac{dg}{dx}\right|\right) &= y.
 \end{align}
 In general, a transform class defines these three operations, from which it is sufficient to perform sampling and scoring.

 The class `pyro.distributions.TransformedDistribution` takes a base distribution of simple noise and a list of transforms, and encapsulates the distribution formed by applying these transformations in sequence. We use it as:

In [ ]:
dist_y = dist.TransformedDistribution(dist_x, [exp_transform])


 Now, plotting samples from both to verify that we that have produced the log-normal distribution:

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(dist_x.sample([1000]).numpy(), bins=50)
plt.title('Standard Normal')
plt.subplot(1, 2, 2)
plt.hist(dist_y.sample([1000]).numpy(), bins=50)
plt.title('Standard Log-Normal')
plt.show()


 Our example uses a single transform. However, we can compose transforms to produce more expressive distributions. For instance, if we apply an affine transformation we can produce the general log-normal distribution,
 \begin{align}
 X &\sim \mathcal{N}(0,1)\\
 Y &= \text{exp}(\mu+\sigma X).
 \end{align}
 or rather, $Y\sim\text{LogNormal}(\mu,\sigma^2)$. In Pyro this is accomplished, e.g. for $\mu=3, \sigma=0.5$, as follows:

In [ ]:
dist_x = dist.Normal(torch.zeros(1), torch.ones(1))
affine_transform = T.AffineTransform(loc=3, scale=0.5)
exp_transform = T.ExpTransform()
dist_y = dist.TransformedDistribution(dist_x, [affine_transform, exp_transform])

plt.subplot(1, 2, 1)
plt.hist(dist_x.sample([1000]).numpy(), bins=50)
plt.title('Standard Normal')
plt.subplot(1, 2, 2)
plt.hist(dist_y.sample([1000]).numpy(), bins=50)
plt.title('Log-Normal')
plt.show()


 For the forward operation, transformations are applied in the order of the list that is the second argument to `TransformedDistribution`. In this case, first `AffineTransform` is applied to the base distribution and then `ExpTransform`.

 ### Learnable Univariate Distributions in Pyro
 Having introduced the interface for invertible transforms and transformed distributions, we now show how to represent *learnable* transforms and use them for density estimation. Our dataset in this section and the next will comprise samples along two concentric circles. Examining the joint and marginal distributions:

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

n_samples = 1000
X, y = datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05)
X = StandardScaler().fit_transform(X)

plt.title(r'Samples from $p(x_1,x_2)$')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.scatter(X[:,0], X[:,1])
plt.show()

plt.subplot(1, 2, 1)
sns.distplot(X[:,0], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
plt.title(r'$p(x_1)$')
plt.subplot(1, 2, 2)
sns.distplot(X[:,1], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
plt.title(r'$p(x_2)$')
plt.show()


 Standard transforms derive from the `pyro.distributions.transforms.Transform` class and are not designed to contain learnable parameters. Learnable transforms, on the other hand, derive from `pyro.distributions.transforms.TransformModule`, which is a `torch.nn.Module` and registers parameters with the object.

 We will learn the marginals of the above distribution using such a transform, `Spline`, defined on a two-dimensional input:


In [ ]:
base_dist = dist.Normal(torch.zeros(2), torch.ones(2))
spline_transform = T.Spline(2, count_bins=16)
flow_dist = dist.TransformedDistribution(base_dist, [spline_transform])


 This transform passes each dimension of its input through a *separate* monotonically increasing function known as a spline. From a high-level, a spline is a complex parametrizable curve for which we can define specific points known as knots that it passes through and the derivatives at the knots. The knots and their derivatives are parameters that can be learnt, e.g., through stochastic gradient descent on a maximum likelihood objective, as we now demonstrate:

In [ ]:
steps = 1000
dataset = torch.tensor(X, dtype=torch.float, requires_grad=True)
optimizer = torch.optim.Adam(spline_transform.parameters(), lr=1e-2)
for step in range(steps):
    optimizer.zero_grad()
    loss = -flow_dist.log_prob(dataset.detach()).mean()
    loss.backward()
    optimizer.step()
    
    if step % 100 == 0:
        print(f'step: {step}, loss: {loss.item()}')


 Plotting samples drawn from the transformed distribution after learning:

In [ ]:
X_flow = flow_dist.sample(torch.Size([1000,])).detach().numpy()
plt.title(r'Joint Distribution')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.scatter(X[:,0], X[:,1], label='data')
plt.scatter(X_flow[:,0], X_flow[:,1], color='firebrick', label='flow')
plt.legend()
plt.show()

plt.subplot(1, 2, 1)
sns.distplot(X[:,0], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='data')
sns.distplot(X_flow[:,0], hist=False, kde=True, 
             bins=None, color='firebrick',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='flow')
plt.title(r'$p(x_1)$')
plt.subplot(1, 2, 2)
sns.distplot(X[:,1], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='data')
sns.distplot(X_flow[:,1], hist=False, kde=True, 
             bins=None, color='firebrick',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='flow')
plt.title(r'$p(x_2)$')
plt.show()


 As we can see, we have learnt close approximations to the marginal distributions, $p(x_1),p(x_2)$. It would have been challenging to fit the irregularly shaped marginals with standard methods, e.g., a mixture of normal distributions. As expected, since there is a dependency between the two dimensions, we do not learn a good representation of the joint, $p(x_1,x_2)$. In the next section, we explain how to learn multivariate distributions whose dimensions are not independent.

 ## Multivariate Distributions
 ### Background
 The fundamental idea of normalizing flows also applies to multivariate random variables, and this is where its value is clearly seen - *representing complex high-dimensional distributions*. In this case, a simple multivariate source of noise, for example a standard i.i.d. normal distribution, $X\sim\mathcal{N}(\mathbf{0},I_{D\times D})$, is passed through a vector-valued bijection, $g:\mathbb{R}^D\rightarrow\mathbb{R}^D$, to produce the more complex transformed variable $Y=g(X)$.

 Sampling $Y$ is again trivial and involves evaluation of the forward pass of $g$. We can score $Y$ using the multivariate substitution rule of integral calculus,
 \begin{align}
     \mathbb{E}_{p_X(\cdot)}\left[f(X)\right] &= \int_{\text{supp}(X)}f(\mathbf{x})p_X(\mathbf{x})d\mathbf{x}\\
     &= \int_{\text{supp}(Y)}f(g^{-1}(\mathbf{y}))p_X(g^{-1}(\mathbf{y}))\det\left|\frac{d\mathbf{x}}{d\mathbf{y}}\right|d\mathbf{y}\\
     &= \mathbb{E}_{p_Y(\cdot)}\left[f(g^{-1}(Y))\right],
 \end{align}
 where $d\mathbf{x}/d\mathbf{y}$ denotes the Jacobian matrix of $g^{-1}(\mathbf{y})$. Equating the last two lines we get,
 \begin{align}
     \ln(p_Y(y)) &= \ln(p_X(g^{-1}(y)))+\ln\left(\det\left|\frac{d\mathbf{x}}{d\mathbf{y}}\right|\right)\\
     &= \ln(p_X(g^{-1}(y)))-\ln\left(\det\left|\frac{d\mathbf{y}}{d\mathbf{x}}\right|\right).
 \end{align}
 Inituitively, this equation says that the density of $Y$ is equal to the density at the corresponding point in $X$ plus a term that corrects for the warp in volume around an infinitesimally small volume around $Y$ caused by the transformation. For instance, in 2 (3) dimensions, the geometric interpretation of the absolute value of the determinant of a Jacobian is that it represents the area (volume) of a cuboid (parallelepiped) with edges defined by the columns of the Jacobian (see *Calculus reference*).

 Similar to the univariate case, we can compose such bijective transformations to produce even more complex distributions. By an inductive argument, if we have $L$ transforms $g_{(0)}, g_{(1)},\ldots,g_{(L-1)}$, then the log-density of the transformed variable $Y=(g_{(0)}\circ g_{(1)}\circ\cdots\circ g_{(L-1)})(X)$ is
 \begin{align}
     \ln(p_Y(y)) &= \ln\left(p_X\left(\left(g_{(L-1)}^{-1}\circ\cdots\circ g_{(0)}^{-1}\right)\left(y\right)\right)\right)+\sum^{L-1}_{l=0}\ln\left(\left|\frac{dg^{-1}_{(l)}(y_{(l)})}{dy'}\right|\right),
     %\left( g^{(l)}(y^{(l)})
     %\right).
 \end{align}
 where we've defined $y_{(0)}=x$, $y_{(L-1)}=y$ for convenience of notation.

 The main challenge is in designing parametrizable multivariate bijections that have closed form expressions for both $g$ and $g^{-1}$, a tractable Jacobian whose calculation scales with $O(D)$ rather than $O(D^3)$, and can express a flexible class of functions.

 ### Multivariate Transforms in Pyro
 Up to this point we have used element-wise transforms in Pyro. These are indicated by having the property `transform.event_dim == 0` set on the transform object. Such element-wise transforms can only be used to represent univariate distributions and multivariate distributions whose dimensions are independent (known in variational inference as the mean-field approximation).

 The power of Normalizing Flow, however, is most apparent in their ability to model complex high-dimensional distributions with neural networks and Pyro contains several such flows for accomplishing this. Transforms that operate on vectors have the property `transform.event_dim == 1`, transforms on matrices with `transform.event_dim == 2`, and so on. In general, the `event_dim` property of a transform indicates how many dependent dimensions there are in the output of a transform.

 In this section, we show how to use `pyro.distributions.transforms.SplineCoupling` to learn the bivariate toy distribution from our running example. A coupling transform (*** references ***) divides the input variable into two parts and applies an element-wise bijection to the section half whose parameters are a function of the first. Optionally, an element-wise bijection is also applied to the first half. Dividing the inputs at $d$, the transform is,
 \begin{align}
 \mathbf{y}_{1:d} &= g_\theta(\mathbf{x}_{1:d})\\
 \mathbf{y}_{(d+1):D} &= h_\phi(\mathbf{x}_{(d+1):D};\mathbf{x}_{1:d}),
 \end{align}
 where $\mathbf{x}_{1:d}$ represents the first $d$ elements of the inputs, $g_\theta$ is either the identity function or an elementwise bijection parameters $\theta$, and $h_\phi$ is an element-wise bijection whose parameters are a function of $\mathbf{x}_{1:d}$.

 This type of transform is easily invertible. We invert the first half, $\mathbf{y}_{1:d}$, then use the resulting $\mathbf{x}_{1:d}$ to evaluate $\phi$ and invert the second half,
 \begin{align}
 \mathbf{x}_{1:d} &= g_\theta^{-1}(\mathbf{y}_{1:d})\\
 \mathbf{x}_{(d+1):D} &= h_\phi^{-1}(\mathbf{y}_{(d+1):D};\mathbf{x}_{1:d}).
 \end{align}
 Difference choices for $g$ and $h$ form different types of coupling transforms. When both are monotonic rational splines, the transform is the spline coupling layer of Neural Spline Flow (*** references ***), which is represented in Pyro by the `pyro.distributions.transforms.SplineCoupling` class. As shown in the references, when we combine a sequence of coupling layers sandwiched between random permutations so we introduce dependencies between all dimensions, we can model complex multivariate distributions. (We will reproduce the state-of-the-art results from those papers in the next tutorial).

 Most of the learnable transforms in Pyro have a corresponding helper function that takes care of constructing a hypernetwork for the transform with the correct output shape. These helper functions are represented by lower-case versions of the corresponding class name, and usually input at the very least the input-dimension or shape of the distribution to model. We create a bivariate flow with a single spline coupling layer as follows:

In [ ]:
base_dist = dist.Normal(torch.zeros(2), torch.ones(2))
spline_transform = T.spline_coupling(2, count_bins=16)
flow_dist = dist.TransformedDistribution(base_dist, [spline_transform])


 Similarly to before, we train this distribution on the toy dataset and plot the results:

In [ ]:
steps = 1000
dataset = torch.tensor(X, dtype=torch.float, requires_grad=True)
optimizer = torch.optim.Adam(spline_transform.parameters(), lr=1e-2)
for step in range(steps):
    optimizer.zero_grad()
    loss = -flow_dist.log_prob(dataset.detach()).mean()
    loss.backward()
    optimizer.step()
    
    if step % 100 == 0:
        print(f'step: {step}, loss: {loss.item()}')

X_flow = flow_dist.sample(torch.Size([1000,])).detach().numpy()
plt.title(r'Joint Distribution')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.scatter(X[:,0], X[:,1], label='data')
plt.scatter(X_flow[:,0], X_flow[:,1], color='firebrick', label='flow')
plt.legend()
plt.show()

plt.subplot(1, 2, 1)
sns.distplot(X[:,0], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='data')
sns.distplot(X_flow[:,0], hist=False, kde=True, 
             bins=None, color='firebrick',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='flow')
plt.title(r'$p(x_1)$')
plt.subplot(1, 2, 2)
sns.distplot(X[:,1], hist=False, kde=True, 
             bins=None,
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='data')
sns.distplot(X_flow[:,1], hist=False, kde=True, 
             bins=None, color='firebrick',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2},
             label='flow')
plt.title(r'$p(x_2)$')
plt.show()


 ## Conditional versus Joint Distributions
 ### Background

 ### Conditional Transforms in Pyro

 ### References
   1. McElreath, D., *Statistical Rethinking, Chapter 7*, 2016
   2. Nunn, N. & Puga, D., *[Ruggedness: The blessing of bad geography in Africa"](https://diegopuga.org/papers/rugged.pdf)*, Review of Economics and Statistics 94(1), Feb. 2012